In [3]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [4]:
import pandas as pd
import numpy as np

In [5]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [6]:
search_queries = [
    "latest SEBI regulations 2023",
    "top mutual funds in India",
    "RBI monetary policy updates",
    "GST changes in India",
    "best Indian stocks to buy",
    "how to invest in Indian real estate",
    "Nifty 50 market analysis",
    "Sensex daily performance",
    "impact of US Fed rates on Indian economy",
    "FDI trends in India",
    "best tax-saving options in India",
    "NPS benefits and drawbacks",
    "gold investment guide India",
    "cryptocurrency regulations in India",
    "start-up funding opportunities in India",
    "IPO calendar India",
    "best digital payment apps in India",
    "financial planning for retirement in India",
    "health insurance comparison India",
    "home loan interest rates forecast India",
    "impact of Brexit on Indian economy",
    "Indian budget highlights",
    "agricultural sector growth in India",
    "forex reserves of India latest",
    "Indian banking sector analysis",
    "e-commerce growth in India",
    "renewable energy investments in India",
    "education loan options in India",
    "automobile industry trends India",
    "MSME financing in India",
    "start-up tax benefits India",
    "best dividend-paying stocks India",
    "real estate market forecast India",
    "IT sector growth in India",
    "impact of COVID-19 on Indian economy",
    "fintech innovations in India",
    "ULIP vs mutual fund comparison",
    "online trading platforms in India",
    "peer-to-peer lending in India",
    "RBI's digital currency plans",
    "demat account opening guide",
    "commodity trading in India",
    "Angel investing in Indian startups",
    "ESG investing in India",
    "top business news India today",
    "venture capital in India",
    "government bonds investment in India",
    "insurance sector reforms India",
    "small-cap stocks in India",
    "portfolio diversification strategies India",
    "M&A trends in Indian market",
    "SIP investment guide",
    "recent changes in Indian tax laws",
    "Indian economic growth forecast",
    "start-up ecosystem in India",
    "private equity in India",
    "corporate governance in Indian companies",
    "inflation rate in India",
    "online business loans in India",
    "SME stock exchange India",
    "pharmaceutical industry in India",
    "tourism industry growth in India",
    "consumer behavior trends India",
    "Indian stock market for beginners",
    "latest FDI policy India",
    "start-up India scheme benefits",
    "public sector banks in India",
    "women entrepreneurs in India",
    "mobile banking trends in India",
    "industrial production index India",
    "balance of trade India",
    "urban infrastructure development India",
    "inward remittances to India",
    "digital marketing for financial services in India",
    "e-wallets market share India",
    "green bonds in India",
    "financial literacy programs in India",
    "Indian rupee exchange rate forecast",
    "personal finance tips for Indians",
    "how to save on taxes in India",
    "latest amendments in Indian Companies Act",
    "mutual fund SIP calculator",
    "credit card usage trends in India",
    "NBFCs in Indian financial market",
    "blockchain technology in Indian banking",
    "RBI guidelines for digital lending",
    "Indian insurance market overview",
    "financial frauds in India",
    "impact of AI on Indian banking",
    "Indian economic policy changes",
    "rural banking in India",
    "stock market trading strategies for Indians",
    "loan against property in India",
    "youth entrepreneurship in India",
    "electronic commerce policy of India",
    "how to invest in Indian bonds",
    "wealth management services in India",
    "cashless transactions in India",
    "start-up valuation methods in India",
    "cybersecurity in Indian banks",
    "forex trading in India"
]

In [7]:
import boto3

In [8]:
sagemaker = boto3.client("sagemaker-runtime", region_name='ap-south-1')

In [10]:
top_tags_file = '../data/top_100_article_tags.json'

In [11]:
import json

In [15]:
with open(top_tags_file,'rb') as f:
    k = json.load(f)

In [20]:
from src._utils import clean_search_query

In [21]:
top_tags = [clean_search_query(x['_id']) for x in k]

In [25]:
top_tags = list(set(top_tags))

In [29]:
gpt_queries = [clean_search_query(x) for x in search_queries]

In [33]:
search_terms = top_tags + gpt_queries

In [9]:
endpoint = 'huggingface-pytorch-inference-2023-10-30-13-16-43-720'

In [28]:
def extract_embeddings_for_text(content: str):
    # embeddings = self.sentence_transformer.encode(content, convert_to_numpy=True)
    # embeddings = list(embeddings)
    # embeddings = [float(x) for x in embeddings]
    # # TODO: - add validation here
    # return embeddings

    input_dct = {"inputs": f"{content}"}
    response = sagemaker.invoke_endpoint(
        EndpointName=endpoint,
        Body=bytes(f'{input_dct}'.replace("'", '"'), 'utf-8'),
        ContentType='application/json'
    )
    vec = response['Body']
    vec = json.loads(vec.read())
    embeddings = vec['vectors'][0]
    # TODO: - add validation here
    return embeddings

In [34]:
embeddings = {}

In [35]:
from tqdm.notebook import tqdm

In [36]:
for txt in tqdm(search_terms):
    embeddings[txt] = extract_embeddings_for_text(content=txt)

  0%|          | 0/194 [00:00<?, ?it/s]

In [38]:
from datetime import datetime

In [39]:
with PostgresDatabaseOperation() as cursor:
    sql = 'INSERT INTO search_query_embeddings (query_text, embedding, model_name, created_at) VALUES (%s, %s, %s, %s)'
    ctime = datetime.now()
    for k in embeddings.keys():
        cursor.execute(sql, (k, embeddings[k], 'BAAI/bge-large-v1.5', ctime))

In [43]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT * FROM search_query_embeddings WHERE query_text = %s'
    cursor.execute(sql, ("sensexsss",))
    results = cursor.fetchall()

In [46]:
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation

In [73]:
def get_query_text_embeddings(query: str):
    with PostgresDatabaseOperation() as cursor:
        sql = 'SELECT embedding FROM search_query_embeddings WHERE query_text = %s LIMIT 1'
        cursor.execute(sql, (query,))
        results = cursor.fetchall()
    if results:
        return json.loads(results[0][0])
    else:
        return None

In [80]:
k = get_query_text_embeddings('sensex')

In [89]:
if '':
    print('a')

In [90]:
a = 'alksjdflksjdlkfj'

In [92]:
a[:1]

'a'

In [87]:
def get_top_1000_articles(emb, k):
    with PostgresDatabaseOperation() as cursor:
        sql = """
        SELECT article_id, 
                   1- (embedding <=> %s) AS distance
            FROM article_embeddings
            ORDER BY distance DESC
            LIMIT %s
        """
        cursor.execute(sql, (str(emb),k))
        results = cursor.fetchall()
    return pd.DataFrame(results, columns = ['article_id', 'dist'])

1 - 1
3 - 0.9
7 - 0.8
14 - 0.7
30 - 0.6
45 - 0.5

In [102]:
def decay_sigmoid_function(n):
    return 0.5 + (0.5/(1 + np.exp((3 * np.log10(n)) - 3)))

In [97]:
def some_exp_fn(n):
    return 1 + np.exp((3 * np.log(n)) - 3)

In [101]:
some_exp_fn(np.e)

2.0

In [103]:
decay_sigmoid_function(3)

0.9137945529934792

In [104]:
from datetime import datetime

In [285]:
df = pd.DataFrame(["2023-05-16T11:29:58+05:30", "2023-10-17T15:11:23+05:30", "2023-11-16T17:09:23+05:30", "2023-10-04T06:30:32+0000"], columns = ['date'])

In [289]:
df['date'] = pd.to_datetime(df['date'],utc=True)

In [290]:
df['date']

0   2023-05-16 05:59:58+00:00
1   2023-10-17 09:41:23+00:00
2   2023-11-16 11:39:23+00:00
3   2023-10-04 06:30:32+00:00
Name: date, dtype: datetime64[ns, UTC]

In [291]:
df['date'] - datetime.today().replace(tzinfo=tzlocal)

0   -185 days +17:54:46.771790
1    -31 days +21:36:11.771790
2     -1 days +23:34:11.771790
3    -44 days +18:25:20.771790
Name: date, dtype: timedelta64[ns]

In [161]:
from datetime import datetime, timedelta

In [159]:
def get_date_string(n_days_ago: int, n_hours_ago: int):
    # Get the current time in UTC and convert it to the desired timezone
    timezone = pytz.timezone('Asia/Kolkata')  # For India Standard Time (UTC+5:30)
    current_time = datetime.now(tz=pytz.utc).astimezone(timezone)
    new_time = current_time - timedelta(days=n_days_ago, hours=n_hours_ago)
    return new_time

In [166]:
pt = {'1': 0.596, '2': 0.807, '3': 1}
cs = {'1': 1, '2': 0.9375, '3': 0.625}

In [6]:
import requests
import json

In [19]:
gst_topic_article = '''
Decoding GST and Other Tax Regulations: A Comprehensive Guide for Indian Financial Enthusiasts

Introduction
Understanding GST and other tax regulations is pivotal for anyone engaged in the Indian financial sector. This article provides insights into the technical nuances and a selection of relevant content to keep you informed and ahead in the field of taxation.

Technical Aspects and Background
To effectively comprehend GST and other tax regulations in India, it's essential to delve into several key areas. Firstly, the Goods and Services Tax (GST), introduced in 2017, revolutionized the indirect taxation system by consolidating multiple taxes into one. Understanding its components—CGST, SGST, IGST—and their applicability across various transactions is crucial.

In addition to GST, being conversant with direct taxes like the Income Tax Act is vital. This encompasses knowledge of corporate taxes, individual income taxes, and various exemptions and deductions. Furthermore, advanced concepts like transfer pricing, international taxation, and Double Taxation Avoidance Agreements (DTAAs) are critical in today's globalized business environment.

Other advanced financial concepts, such as financial derivatives (F&Os), Black-Scholes models for option pricing, and comprehensive strategies for company valuation, also intersect with tax planning and compliance.

Common Examples of Topics
In this field, one would frequently encounter topics such as:

GST Compliance and Return Filing: Navigating through the complexities of GST filing procedures and deadlines.
Input Tax Credit in GST: Understanding how to optimize tax credits against GST liabilities.
Corporate Taxation and Planning: Strategies for minimizing tax liabilities within legal frameworks.
International Taxation and DTAA: The impact of international tax laws and agreements on Indian entities.
Transfer Pricing Regulations: Managing cross-border transactions within related entities.
Types of Content and Article Titles
News and Updates

"GST Council's Latest Amendments: A Closer Look"
"New Direct Tax Code: Implications for Corporate India"
"India's Stance in the Global Taxation Debate: Recent Developments"
"Transfer Pricing: RBI's Latest Guidelines"
"E-invoicing Under GST: A Game Changer for Tax Compliance?"
Opinion and Analysis

"Is GST Fulfilling Its Promise of a Unified Tax System?"
"Decoding India's Tax Structure: Are We Competitive Enough?"
"Evaluating the Impact of International Tax Laws on Indian Economy"
"Corporate Tax Cuts: Boon or Bane for Economic Growth?"
"Navigating Transfer Pricing: Balancing Compliance and Efficiency"
Educational and How-To Guides

"A Step-by-Step Guide to Filing Your GST Returns"
"Understanding Your Rights Around Input Tax Credit"
"Demystifying Corporate Tax Planning for Indian Businesses"
"International Taxation: How It Affects You and Your Business"
"Mastering Transfer Pricing: Strategies and Compliance"
This comprehensive overview provides the foundational knowledge and resources necessary for anyone keen on understanding and keeping abreast of GST and other tax regulations in India. With these insights, readers can navigate the complex landscape of Indian taxation with greater ease and proficiency.
'''

In [26]:
gst_2 = '''
GST Compliance, Tax Legislation, VAT Regulations, Excise Duty Policy, Corporate Tax Rules, Tax Evasion Penalties, Customs Duty Guidelines, Service Tax Framework, GST Returns Filing, Tax Audit Procedures, International Taxation Laws, E-Invoicing Standards, Indirect Tax Strategies, Taxation of Digital Goods, GST Rate Structure, Tax Incentives, Tax Exemption Criteria, Fiscal Policy, Tax Rebate Schemes, Transfer Pricing Regulations, Tax Dispute Resolution, Taxation of E-Commerce, Cross-Border Taxation, Input Tax Credit, Sales Tax Norms, GST Registration, Tax Deductions, Tax Havens, Wealth Tax, Tax Planning Strategies, Environmental Taxation, Value-Added Tax (VAT) Compliance, Taxation of Cryptocurrencies, Property Tax Laws, Tax Accounting Standards, Consumption Taxes, Taxation of Non-Profits, Tax Treaty Agreements, Payroll Tax Obligations, Taxation of Dividends, Inheritance Tax Regulations, Indirect Tax Compliance, Tax Collection Methods, Taxation of Investments, Tax Reporting Requirements, GST Audit Framework, Tax Evasion Laws, Income Tax Slabs, Taxation of Freelancers.
'''

In [4]:
gst_3 = '''
GST Compliance, Tax Reforms, GST Returns, Input Tax Credit, GST Slabs, E-way Bills, GST Council, Direct Tax, GST Rate Changes, Tax Evasion, GSTN Portal, Tax Policy, Indirect Taxes, GST Registration, VAT Integration, GST Amendments, Tax Deductions, Corporate Tax, GST Legislation, HSN Code, Tax Audits, GST Exemptions, Digital Tax, GST Collection, Customs Duty, Tax Litigation, GST Implementation, Income Tax, Tax Incentives, Fiscal Policy, GST Transition, Tax Compliance, Service Tax, Tax Planning, Excise Duty, GST Framework, GST Invoice, Tax Penalties, Sales Tax, GST Challenges, Tax Avoidance, GST Impact, Tax Governance, Financial Regulations, GST Education, Tax Transparency, Capital Gains Tax, Business Taxation, Tax Administration.
'''

In [2]:
import requests, json

In [5]:
r1 = requests.post('http://Hubble-env.eba-pguqm8s4.ap-south-1.elasticbeanstalk.com/search_query', json = {'query_text': gst_3})

In [40]:
json.loads(r.text)['additionalInfo']['6551c9904b13023f9348b421']

{'actual_similarity': 0.6087753956887371,
 'avg_score': 0.8677048909487683,
 'num_days_ago': 3,
 'recency_score': 0.914,
 'scaled_similarity': 0.8445573364231526}

In [8]:
k = {'meta_data': {'is_premium_article': False, 'title': 'US Secy Blinken highlights civilian suffering, security concerns amid Israel-Hamas war', 'short_description': "US Secretary of State Antony Blinken's visit to Israel during the ongoing Israel-Hamas conflict has shed light on the horrific realities faced by civilians in the region. Blinken expressed anguish at the barbarity of the Hamas attack and emphasized that civilians are not the target of Israel's operations. He discussed efforts to address the humanitarian needs of the people in Gaza while safeguarding them from harm and reaffirmed the US's unwavering support for Israel's right to defend its people.", 'published_time': '2023-10-13T07:17:00+05:30', 'last_updated_time': '2023-10-13T07:20:00+05:30', 'tags': ['israel', 'Tel Aviv', 'Hamas attack', 'Israel support', 'Antony Blinken', 'US Secretary', 'joe biden', 'Benjamin Netanyahu'], 'image_url': 'https://economictimes.indiatimes.com/thumb/msid-104385094,width-600,height-450,imgsize-11566,overlay-economictimes/photo.jpg', 'authors': ['ANI']}, 'cleaned_text': "US Secretary of State Antony Blinken's visit to Israel amid the ongoing Israel-Hamas war has brought to light the harrowing realities faced by civilians in the region, as well as the security concerns that continue to escalate.  I've seen a baby, an infant riddled with bullets. Soldiers beheaded. Young people burned alive in their cars or in their highway rooms. It almost defies comprehension,  Secretary Blinken expressed anguish at the barbarity of the Hamas attack on October 7 in southern Israel. So, we did see, photographs and videos that, the Israeli government shared with us... It's hard to find the right words. It's beyond what anyone would ever wanna imagine much less actually see and god forbid experience,  said Blinken on horrific photographs of the Hamas attack.He also shed light on the challenges posed by Hamas' use of civilians as human shields, which has been a recurring and deeply troubling tactic. Blinken emphasised that civilians are not the target of Israel's operations in Gaza. On the Mediterranean situation in Gaza, I think it's first important to remember a fundamental issue that makes this complicated. Hamas continues to use civilians as human shields. Something that's not new, something that they've always done intentionally putting, civilians in harm's way to protect them to try to protect themselves or protect their structure or protect their weapons... Civilians should not be used in any way as the targets of military operations. They are not the target of Israel's operations,  added the US Secretary.Furthermore, Secretary Blinken discussed efforts to address the humanitarian needs of the people living in Gaza while safeguarding them from harm during Israel's legitimate security operations aimed at defending itself from terrorism. He also explored the possibilities of providing safe passage for civilians who wish to leave Gaza, acknowledging the importance of protecting innocent lives. We did discuss ways to address the humanitarian needs of people living in, Gaza to protect them from harm while Israel conducts its legitimate security operations to defend itself from terrorism and to try to ensure that this never happens again. We also talked about, possibilities for safe passage for civilians who want to leave or get out of the way, in Gaza, and that's, a conversation to the discussion that we will pursue, in the coming days, including some of the countries that we'll be visiting...,  he said.In his discussions with Israeli leadership, Secretary Blinken reaffirmed the United States' unwavering support for Israel's right to defend its people and its determination to prevent a recurrence of the tragic events that unfolded recently. Secretary Blinken, on behalf of President Biden, also emphasized the importance of preventing any further fronts in the ongoing conflict and warned against any attempts by state or non-state actors to exploit the situation. It's our determination and, that of Israel as well, that not be a second front or a third front. And we are working, as hard as we can, working with other partners in the region, to try, to ensure that the case. President Biden has also been very clear that no one, state or non-state actor should try to take advantage of this moment...,  Blinken said.The US is actively assisting Israel at its request and is committed to providing the necessary support for the nation's security efforts. President's been very clear we stand with Israel. We stand with Israel in its determination to defend its people, defend its country. We stand with Israel in its determination to do everything possible to ensure that what happened on Saturday never happens again,  he added.Earlier in the day, Israeli President Isaac Herzog thanked Blinken and the US for their continuous support and added that the security and defence of the Israeli people is the highest priority at the moment. One thing is clear. Because of that day - it's the highest number of Jews ever killed since the Holocaust - I think we must be clear that this is not only unacceptable, which the president said, but the sense of security and defence that the people of Israel deserve is something which is of highest priority so that these pictures will never occur,  Herzog said.Blinken also met Prime Minister Benjamin Netanyahu and affirmed US' commitment to support Israel. The message that I bring to Israel is this -- you may be strong enough on your own to defend yourself, but as long as America exists, you will never, ever have to. We will always be there by your side,  The Times of Israel quoted Blinken as saying.He added,  That's the message that President Joe Biden delivered to the prime minister from the moment that this crisis began .Notably, the death toll in Israel since the Hamas attack on October 7 has jumped to 1,300 and some 3300 have been injured, including 28 in critical condition and 350 in serious condition, The Times of Israel reported citing Hebrew media reports.The fate of an estimated 150 people abducted and taken to the Gaza Strip during the attack by Hamas is still unclear, the report said.", 'article_images': [{'image_url': 'https://img.etimg.com/thumb/msid-104385094,width-300,height-225,imgsize-11566,,resizemode-75/us-secy-blinken-highlights-civilian-suffering-security-concerns-amid-israel-hamas-war.jpg', 'image_caption': 'US Secy Blinken highlights civilian suffering, security concerns amid Israel-Hamas war'}]}

In [10]:
k1 = {'url': 'https://economictimes.indiatimes.com/news/international/world-news/us-secy-blinken-highlights-civilian-suffering-security-concerns-amid-israel-hamas-war/articleshow/104385083.cms', 'published_time': '2023-10-13T07:17:00+05:30', 'source_id': '6512cdcad01a9c8e86263e05', 'title': 'US Secy Blinken highlights civilian suffering, security concerns amid Israel-Hamas war', 'article_id': '6528b2d850af0e25a9122f54', 'cleaned_text': '', 'is_premium_article': False}

In [11]:
k.update(k1)

In [12]:
k

{'meta_data': {'is_premium_article': False,
  'title': 'US Secy Blinken highlights civilian suffering, security concerns amid Israel-Hamas war',
  'short_description': "US Secretary of State Antony Blinken's visit to Israel during the ongoing Israel-Hamas conflict has shed light on the horrific realities faced by civilians in the region. Blinken expressed anguish at the barbarity of the Hamas attack and emphasized that civilians are not the target of Israel's operations. He discussed efforts to address the humanitarian needs of the people in Gaza while safeguarding them from harm and reaffirmed the US's unwavering support for Israel's right to defend its people.",
  'published_time': '2023-10-13T07:17:00+05:30',
  'last_updated_time': '2023-10-13T07:20:00+05:30',
  'tags': ['israel',
   'Tel Aviv',
   'Hamas attack',
   'Israel support',
   'Antony Blinken',
   'US Secretary',
   'joe biden',
   'Benjamin Netanyahu'],
  'image_url': 'https://economictimes.indiatimes.com/thumb/msid-1043

In [9]:
k

{'meta_data': {'is_premium_article': False,
  'title': 'US Secy Blinken highlights civilian suffering, security concerns amid Israel-Hamas war',
  'short_description': "US Secretary of State Antony Blinken's visit to Israel during the ongoing Israel-Hamas conflict has shed light on the horrific realities faced by civilians in the region. Blinken expressed anguish at the barbarity of the Hamas attack and emphasized that civilians are not the target of Israel's operations. He discussed efforts to address the humanitarian needs of the people in Gaza while safeguarding them from harm and reaffirmed the US's unwavering support for Israel's right to defend its people.",
  'published_time': '2023-10-13T07:17:00+05:30',
  'last_updated_time': '2023-10-13T07:20:00+05:30',
  'tags': ['israel',
   'Tel Aviv',
   'Hamas attack',
   'Israel support',
   'Antony Blinken',
   'US Secretary',
   'joe biden',
   'Benjamin Netanyahu'],
  'image_url': 'https://economictimes.indiatimes.com/thumb/msid-1043

In [7]:
json.loads(r1.text)['searchArticleIds']

[{'article_id': '65367fe21e5cc42b1b143c76'},
 {'article_id': '65316e121e5cc42b1b13e420'},
 {'article_id': '651dd02ca662d76276b7d165'},
 {'article_id': '653169051e5cc42b1b13c998'},
 {'article_id': '653169151e5cc42b1b13ce74'},
 {'article_id': '6531690d1e5cc42b1b13cbfe'},
 {'article_id': '653169141e5cc42b1b13ce10'},
 {'article_id': '65316fed1e5cc42b1b1428c9'},
 {'article_id': '65438140dc4fa72a6c403871'},
 {'article_id': '6525a763598c6618f0608c4a'},
 {'article_id': '651e016da662d76276b84fa0'},
 {'article_id': '651ddb0aa662d76276b7edea'},
 {'article_id': '651e0b37a662d76276b86742'},
 {'article_id': '653170071e5cc42b1b143383'},
 {'article_id': '65325dd61e5cc42b1b1438a9'},
 {'article_id': '65325de31e5cc42b1b1438c0'},
 {'article_id': '65316efc1e5cc42b1b13eb75'},
 {'article_id': '653170011e5cc42b1b14312e'},
 {'article_id': '651e99e6a662d76276b8a7aa'},
 {'article_id': '6541f78edc4fa72a6c40373c'},
 {'article_id': '651e1f44a662d76276b8962e'},
 {'article_id': '6531700c1e5cc42b1b14359a'},
 {'article

In [201]:
a = [1,2,3]

In [202]:
a[:100]

[1, 2, 3]

In [247]:
df['date'] = pd.to_datetime(df['date'])

In [123]:
import pytz

In [151]:
from dateutil import tz

In [152]:
19800/3600

5.5

In [221]:
tzlocal = tz.tzoffset('IST', 19800)

In [222]:
#timezone = pytz.timezone('Asia/Kolkata')

In [223]:
datetime.today().replace(tzinfo=tzlocal)

datetime.datetime(2023, 11, 16, 13, 55, 45, 399941, tzinfo=tzoffset('IST', 19800))

In [243]:
df['date']

0   2023-05-16 11:29:58+05:30
1   2023-10-17 15:11:23+05:30
2   2023-11-15 13:11:23+05:30
Name: date, dtype: datetime64[ns, UTC+05:30]

In [253]:
datetime.today().replace(tzinfo=tzlocal)

datetime.datetime(2023, 11, 16, 17, 10, 12, 444691, tzinfo=tzoffset('IST', 19800))

In [256]:
(datetime.today().replace(tzinfo=tzlocal) - df['date']).dt.days

0    184
1     30
2      0
Name: date, dtype: int64

In [257]:
df['n'] = ( df['date']).dt.days

In [261]:
[(datetime.today().replace(tzinfo=tzlocal) - x).days for x in df['date']]

[184, 30, 0]

In [258]:
df

,date,n
0,2023-05-16 11:29:58+05:30,184
1,2023-10-17 15:11:23+05:30,30
2,2023-11-16 17:09:23+05:30,0


In [180]:
def decay_sigmoid_function(n):
    return 0.5 + (0.5/(1 + np.exp((3 * np.log10(n)) - 3)))

In [185]:
decay_sigmoid_function(6)

0.8302525868073324

In [220]:
timezone

<DstTzInfo 'Asia/Kolkata' LMT+5:53:00 STD>

In [229]:
def get_date_string(n_days_ago: int, n_hours_ago: int = 0):
    # Get the current time in UTC and convert it to the desired timezone
    current_time = datetime.today().replace(tzinfo=tzlocal)
    new_time = current_time - timedelta(days=n_days_ago, hours=n_hours_ago)
    date_string = new_time.strftime('%Y-%m-%dT%H:%M:%S%z')
    return date_string

In [233]:
published_time_df = pd.DataFrame([('1', get_date_string(n_days_ago=30)),
                                          ('2', get_date_string(n_days_ago=7)),
                                          ('3', get_date_string(n_days_ago=0, n_hours_ago=12))], columns=['article_id', 'published_time'])

In [234]:
published_time_df

,article_id,published_time
0,1,2023-10-17T13:56:22+0530
1,2,2023-11-09T13:56:22+0530
2,3,2023-11-16T01:56:22+0530


In [235]:
published_time_df.dtypes

article_id        object
published_time    object
dtype: object

In [236]:
def get_published_time_scores(published_time_df):
    published_time_df['published_time'] = pd.to_datetime(published_time_df['published_time'])
    today_reference = datetime.today().replace(tzinfo=timezone)
    published_time_df['num_days_ago'] = (today_reference - published_time_df['published_time']).dt.days
    published_time_df['recency_score'] = [compute_publication_decay_time(n) for n in published_time_df['num_days_ago']]
    return published_time_df

In [191]:
def compute_publication_decay_time(num_days_ago: int):
    assert num_days_ago >= 0 and isinstance(num_days_ago, int), f"num_days_ago should be an int > 1 and not {num_days_ago}"
    if not decay_values:
        for i in range(0, 61):
            decay_values[i] = decay_sigmoid_function(n=i)
    if num_days_ago >= 61:
        return 0.5
    else:
        return decay_values[num_days_ago]

In [192]:
def decay_sigmoid_function(n):
    return 0.5 + (0.5/(1 + np.exp((3 * np.log10(n)) - 3)))


decay_values = {}
for i in range(0, 61):
    decay_values[i] = np.round(decay_sigmoid_function(n=i), 3)

/var/folders/d4/cgyr_gnj7nn2wy_hq40gkq8c0000gq/T/ipykernel_96853/1403354633.py:2: RuntimeWarning: divide by zero encountered in log10
  return 0.5 + (0.5/(1 + np.exp((3 * np.log10(n)) - 3)))


In [238]:
get_published_time_scores(published_time_df)

,article_id,published_time,num_days_ago,recency_score
0,1,2023-10-17 13:56:22+05:30,29,0.60
1,2,2023-11-09 13:56:22+05:30,6,0.83
2,3,2023-11-16 01:56:22+05:30,0,1.00


In [177]:
def compute_decay(n):
    if n <= 7:
        return 1
    elif n <= 45:
        return 0.7
    else:
        return 0.5

In [178]:
df['decay'] = [compute_decay(k) for k in df['n']]

In [179]:
df

,date,n,decay
0,2023-05-16 11:29:58+05:30,184,0.5
1,2023-10-17 15:11:23+05:30,30,0.7
2,2023-11-15 13:11:23+05:30,1,1.0


In [105]:
datetime.today()

datetime.datetime(2023, 11, 15, 18, 10, 6, 112437)

In [88]:
get_top_1000_articles(k, 20)

,article_id,dist
0,6540c74c2936d70acf71e563,0.748928
1,6540c7502936d70acf71e728,0.748928
2,6540c7512936d70acf71e74a,0.748928
3,6540c74f2936d70acf71e664,0.748928
4,6540c74d2936d70acf71e5b3,0.748928
5,6540c7512936d70acf71e740,0.748928
6,6540c7572936d70acf71ea03,0.748928
7,6540c75b2936d70acf71eb96,0.748928
8,6540c75a2936d70acf71eb49,0.748928
9,6540c75a2936d70acf71eb6a,0.748928


In [71]:
%timeit json.loads(k[0][0])

61.9 µs ± 837 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [72]:
%timeit ast.literal_eval(k[0][0])

1.19 ms ± 15.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [65]:
import ast

In [67]:
ss = ast.literal_eval(k[0][0])

In [68]:
ss[0]

-0.1486416

In [61]:
[float(x) for x in k[0][0]]

ValueError: could not convert string to float: '['

In [53]:
k[0]

TypeError: 'NoneType' object is not subscriptable

In [ ]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT embedding FROM search_query_embeddings WHERE query_text = %s LIMIT 1'
    cursor.execute(sql, (query,))
    results = cursor.fetchall()
if results:
    return results[0]
else:
    return None

In [44]:
results

[]